In [1]:
import pandas as pd
import numpy as np
import re
#import tldextract

In [2]:
story6y = pd.read_csv("6yn.csv", index_col=0)

/home/pding/anaconda3/envs/tdi/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
story6y

,title,text,timestamp,url,score,dead,deleted
0,How to Not Mess Up Your Tech Startup,NaN,2013-06-01 00:01:56+00:00,http://kristinabjoran.com/how-to-not-mess-up-y...,2.0,NaN,NaN
1,NaN,NaN,2013-06-01 00:03:32+00:00,NaN,NaN,True,True
2,Someday you may ditch your two-factor authenti...,NaN,2013-06-01 00:04:34+00:00,http://arstechnica.com/security/2013/05/someda...,1.0,NaN,NaN
3,NaN,NaN,2013-06-01 00:05:35+00:00,NaN,NaN,True,True
4,Will State.com become the world's opinion forum?,NaN,2013-06-01 00:05:54+00:00,http://pandodaily.com/2013/05/31/making-sense-...,2.0,NaN,NaN
...,...,...,...,...,...,...,...
2344431,Italy Follows France in Levying a Digital Tax,NaN,2019-12-29 23:53:49+00:00,https://www.wsj.com/articles/italy-follows-fra...,2.0,NaN,NaN
2344432,Amazon is looking into tech that can identify ...,NaN,2019-12-29 23:53:51+00:00,https://www.usatoday.com/story/tech/2019/12/27...,1.0,NaN,NaN
2344433,Giant Concentric Circles in Granite Springs Va...,NaN,2019-12-29 23:54:04+00:00,https://www.blackrockdesert.org/wiki/index.php...,2.0,NaN,NaN
2344434,Wyze Data leak 12-26-2019,NaN,2019-12-29 23:55:59+00:00,https://forums.wyzecam.com/t/updated-12-29-19-...,1.0,NaN,NaN


In [4]:
# replace non values with nan
story6y.fillna(value=np.nan, inplace=True)
# filter out delted stories
story6y_f = story6y[story6y.deleted.isnull()]

In [5]:
# build the series of content, ignoring nan values
stories = story6y_f.title.fillna(' ').astype(str) + ' ' + story6y_f.text.fillna(' ').astype(str)

In [6]:
stories

0                     How to Not Mess Up Your Tech Startup  
2          Someday you may ditch your two-factor authenti...
4           Will State.com become the world's opinion for...
5          LightUp Teaches Kids Electronics With Augmente...
6          Getting Things Done: Why GTD for Sales is the ...
                                 ...                        
2344431      Italy Follows France in Levying a Digital Tax  
2344432    Amazon is looking into tech that can identify ...
2344433    Giant Concentric Circles in Granite Springs Va...
2344434                          Wyze Data leak 12-26-2019  
2344435    4M cards,4K drawers–coalition of book lovers r...
Length: 2243679, dtype: object

In [7]:
# extract domain as an additional feature
domains = story6y_f.url.fillna(' ')

In [8]:
maindomain = domains.apply(lambda x: tldextract.extract(x).domain)

In [9]:
top15sites = maindomain.value_counts()[1:16]

In [10]:
top15sites

github         104365
medium          93788
youtube         55132
nytimes         39535
techcrunch      26840
blogspot        23918
bbc             22963
theguardian     19916
arstechnica     19334
google          17919
bloomberg       17633
wikipedia       17264
theverge        15273
wsj             14189
wired           14165
Name: url, dtype: int64

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [12]:
# text prepprocessing
lemmatizer=WordNetLemmatizer() #For words Lemmatization
stop_words=set(stopwords.words('english'))

In [13]:
# text processing
# 1. convert to lower case, clean text of all punctuations
# 2. remove stop words
# 3. lemmatize

def process_text(text):
    tokenized_text = word_tokenize(text.lower())
    cleaned_text = ' '.join([lemmatizer.lemmatize(t) for t in tokenized_text if t not in stop_words and re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', t)])
    return cleaned_text


In [14]:
stories_clean = stories.apply(lambda x: process_text(x))

In [ ]:
top15words = stories_clean.apply(lambda x: x.split()).value_counts()[1:16]

In [15]:
stories_clean

0                                          mess tech startup
2          someday may ditch two-factor authenticator ele...
4                       state.com become world opinion forum
5            lightup teach kid electronics augmented reality
6                         getting thing done gtd sale answer
                                 ...                        
2344431             italy follows france levying digital tax
2344432         amazon looking tech identify using vein hand
2344433        giant concentric circle granite spring valley
2344434                                       wyze data leak
2344435    cards,4k drawers–coalition book lover rush sav...
Length: 2243679, dtype: object

In [16]:
# add main domain name back to stories
stories_aug = stories_clean + ' ' + maindomain

In [17]:
stories_aug

0                           mess tech startup kristinabjoran
2          someday may ditch two-factor authenticator ele...
4            state.com become world opinion forum pandodaily
5          lightup teach kid electronics augmented realit...
6              getting thing done gtd sale answer voltagecrm
                                 ...                        
2344431         italy follows france levying digital tax wsj
2344432    amazon looking tech identify using vein hand u...
2344433    giant concentric circle granite spring valley ...
2344434                               wyze data leak wyzecam
2344435    cards,4k drawers–coalition book lover rush sav...
Length: 2243679, dtype: object

In [18]:
stories_aug.to_csv('saug.csv')

In [19]:
# As discussed in the first version, an auto encoder should be built here to filter out low quality posts

# 

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import random
import math
import time

In [4]:
# ensure reproductivity
SEED = 6745

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [4]:
# Load embedding files


# import the embeddings
EMBEDDING_FILE = '/home/pding/DSP/embed/glove.840B.300d.txt'

# Embedding file organized as word, followed by the vector
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8"))



In [27]:
dict(list(embeddings_index.items())[0:7])

{',': array([-0.082752 ,  0.67204  , -0.14987  , -0.064983 ,  0.056491 ,
         0.40228  ,  0.0027747, -0.3311   , -0.30691  ,  2.0817   ,
         0.031819 ,  0.013643 ,  0.30265  ,  0.0071297, -0.5819   ,
        -0.2774   , -0.062254 ,  1.1451   , -0.24232  ,  0.1235   ,
        -0.12243  ,  0.33152  , -0.006162 , -0.30541  , -0.13057  ,
        -0.054601 ,  0.037083 , -0.070552 ,  0.5893   , -0.30385  ,
         0.2898   , -0.14653  , -0.27052  ,  0.37161  ,  0.32031  ,
        -0.29125  ,  0.0052483, -0.13212  , -0.052736 ,  0.087349 ,
        -0.26668  , -0.16897  ,  0.015162 , -0.0083746, -0.14871  ,
         0.23413  , -0.20719  , -0.091386 ,  0.40075  , -0.17223  ,
         0.18145  ,  0.37586  , -0.28682  ,  0.37289  , -0.16185  ,
         0.18008  ,  0.3032   , -0.13216  ,  0.18352  ,  0.095759 ,
         0.094916 ,  0.008289 ,  0.11761  ,  0.34046  ,  0.03677  ,
        -0.29077  ,  0.058303 , -0.027814 ,  0.082941 ,  0.1862   ,
        -0.031494 ,  0.27985  , -0.074412 ,

In [28]:
# tf.tokenizer removes all the punct
# numbers are represented as #, ##, ..(according to their length) in glove
def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

In [29]:
traintitle = stories_aug.apply(lambda x: clean_numbers(x))

In [31]:
traintitle

0                           mess tech startup kristinabjoran
2          someday may ditch two-factor authenticator ele...
4            state.com become world opinion forum pandodaily
5          lightup teach kid electronics augmented realit...
6              getting thing done gtd sale answer voltagecrm
                                 ...                        
2344431         italy follows france levying digital tax wsj
2344432    amazon looking tech identify using vein hand u...
2344433    giant concentric circle granite spring valley ...
2344434                               wyze data leak wyzecam
2344435    cards,4k drawers–coalition book lover rush sav...
Length: 2243679, dtype: object

In [5]:
#traintitle.to_csv('traintae.csv')
traintitle = pd.read_csv("traintae.csv", index_col=0)

/home/pding/anaconda3/envs/tdi/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
traintitle = traintitle['0']

In [7]:


## Tokenize the sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(traintitle))
train_X = tokenizer.texts_to_sequences(traintitle)
word_index = tokenizer.word_index


In [7]:
len(word_index)

430422

In [38]:
lengths = [len(i) for i in train_X]

In [41]:
pd.DataFrame(lengths).describe()

,0
count,2.243679e+06
mean,8.971123e+00
std,1.498883e+01
min,0.000000e+00
25%,5.000000e+00
50%,6.000000e+00
75%,8.000000e+00
max,2.113000e+03


In [8]:
# We see that most of the titles+texts have very little words, and the longest posts are much longer than the average ones. This is probably
# Because most of the posts only contain a title and no other contents
maxlen = 10

In [9]:
## Pad the sentences (same length input), texts_to_matrix would be too large
train_X = pad_sequences(train_X, maxlen=maxlen)
train_X = pad_sequences(train_X, maxlen=maxlen)

In [10]:
train_X.shape

(2243679, 10)

In [44]:
train_X[0:5]

array([[     0,      0,      0,      0,      0,      0,   2884,     46,
            16, 213854],
       [     0,  10945,    147,   3978,    175,   1269,  10588,   1923,
          5339,    121],
       [     0,      0,      0,    182,     13,    362,     36,    958,
          1229,   6815],
       [     0,      0,      0, 146017,    928,    397,   2123,   2188,
           503,     63],
       [     0,      0,      0,    180,     54,    403,  11329,    228,
           531, 213855]], dtype=int32)

In [11]:
train_X = torch.as_tensor(train_X, dtype=torch.long)

In [12]:
# use local rtx 2080
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
# prepare the embedding matrix
# load the embeddings
all_embs = np.stack(list(embeddings_index.values()))
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]
nb_words = len(word_index)

# build embedding matrix, for words not in glove use random vectors
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))

# build embedding matrix
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [21]:
embedding_matrix.shape

(430422, 300)

In [73]:
embedwts = torch.tensor(embedding_matrix, dtype=torch.float32, device=device)

In [13]:
# build input iterator, create minibatches
trn = {'batch_size': 256,
          'shuffle': True}
evl = {'batch_size': 256,
          'shuffle': False}

trainiter = DataLoader(train_X, **trn)
#next(iter(data_loader))

In [14]:
# Define the autoencoder model separately, wiht endocer, decoder and the whole model

class Enc0(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout, batch_first=True)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, tin):
        
        
        embedded = self.dropout(self.embedding(tin))        
       
        outputs, (hidden, cell) = self.rnn(embedded)
              
        return embedded, outputs, hidden, cell

# in this version use the pretrained embeddings
class Enc1(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout, batch_first=True)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, tin):
        
        
        embedded = self.dropout(self.embedding(tin))        
       
        outputs, (hidden, cell) = self.rnn(embedded)
              
        return embedded, outputs, hidden, cell


class Dec0(nn.Module):
    def __init__(self, output_dim, input_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
             
        self.rnn = nn.LSTM(input_dim, hid_dim, n_layers, dropout = dropout, batch_first=True)
        
        #self.fc_out = nn.Linear(hid_dim, output_dim)
        
    def forward(self, zve, hidden, cell):
                
        output, (hidden, cell) = self.rnn(zve, (hidden, cell))
        
        #use the output as the basis of recontruction
        prediction = output
        
        return prediction
    
# add a dense layer to the decoder, so that the hid_dim does not need to be 300

class Dec1(nn.Module):
    def __init__(self, output_dim, input_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
             
        self.rnn = nn.LSTM(input_dim, hid_dim, n_layers, dropout = dropout, batch_first=True)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
    def forward(self, zve, hidden, cell):
                
        output, (hidden, cell) = self.rnn(zve, (hidden, cell))
        
        #use the output as the basis of recontruction
        prediction = self.fc_out(output)
        
        return prediction

class AE0(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, tin, maxlen):
        embedded, outputs, hidden, cell = self.encoder(tin)
        # replicate outputs as the new input to the decoer model
        hiddeni = hidden.transpose(1,0).repeat(1,maxlen,1).to(self.device)
        #hiddeni = hidden.repeat(1,maxlen, 1)
        recons = self.decoder(hiddeni, hidden, cell)
        
        return embedded, recons      

In [15]:
INPUT_DIM = maxlen
N_EMB = 430422+1
#0 is a reserved index that won't be assigned to any word.
EMB_DIM = 300
OUTPUT_DIM = 300
HID_DIM_E = 20
HID_DIM_D = 20
N_LAYERS = 1
ENC_DROPOUT = 0
DEC_DROPOUT = 0

enc = Enc0(N_EMB, EMB_DIM, HID_DIM_E, N_LAYERS, ENC_DROPOUT)
dec = Dec1(OUTPUT_DIM, HID_DIM_E, HID_DIM_D, N_LAYERS, DEC_DROPOUT)

model0 = AE0(enc, dec, device).to(device)

In [16]:
optimizer = optim.Adam(model0.parameters())
criterion  = nn.MSELoss()

In [17]:
def train(model, iterator, optimizer, criterion, clip, device, maxlen):
    model.train()
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        tin = batch.to(device)
        
        optimizer.zero_grad()
        
        target, recons = model(tin, maxlen)
        
        loss = criterion(recons, target)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()

        #print(f'Batch：{i+1} | Loss: {loss.item()}')
        
    return epoch_loss / len(iterator)

In [15]:
CLIP = 1
train(model0, trainiter, optimizer, criterion, CLIP,  device, maxlen)

Batch：1 | Loss: 1.02158522605896
Batch：2 | Loss: 1.019858479499817
Batch：3 | Loss: 1.0142536163330078
Batch：4 | Loss: 1.0087846517562866
Batch：5 | Loss: 1.0112082958221436
Batch：6 | Loss: 1.0088019371032715
Batch：7 | Loss: 1.0052598714828491
Batch：8 | Loss: 1.0015226602554321
Batch：9 | Loss: 1.0041788816452026
Batch：10 | Loss: 0.9973664283752441
Batch：11 | Loss: 0.9957917928695679
Batch：12 | Loss: 0.9938660264015198
Batch：13 | Loss: 0.9925559759140015
Batch：14 | Loss: 0.9886940717697144
Batch：15 | Loss: 0.9843347668647766
Batch：16 | Loss: 0.9855119585990906
Batch：17 | Loss: 0.9815331697463989
Batch：18 | Loss: 0.984406590461731
Batch：19 | Loss: 0.9788204431533813
Batch：20 | Loss: 0.9729545712471008
Batch：21 | Loss: 0.9689456224441528
Batch：22 | Loss: 0.968106746673584
Batch：23 | Loss: 0.9678791165351868
Batch：24 | Loss: 0.9569656252861023
Batch：25 | Loss: 0.95974200963974
Batch：26 | Loss: 0.9613344073295593
Batch：27 | Loss: 0.9561877250671387
Batch：28 | Loss: 0.9529245495796204
Batch：29

Batch：229 | Loss: 0.5979296565055847
Batch：230 | Loss: 0.6030871868133545
Batch：231 | Loss: 0.6086776852607727
Batch：232 | Loss: 0.6104727387428284
Batch：233 | Loss: 0.5759015083312988
Batch：234 | Loss: 0.6116357445716858
Batch：235 | Loss: 0.6134580373764038
Batch：236 | Loss: 0.6119598150253296
Batch：237 | Loss: 0.6121341586112976
Batch：238 | Loss: 0.5693818926811218
Batch：239 | Loss: 0.5825315713882446
Batch：240 | Loss: 0.5559849143028259
Batch：241 | Loss: 0.5999267101287842
Batch：242 | Loss: 0.600415825843811
Batch：243 | Loss: 0.5857189893722534
Batch：244 | Loss: 0.5993841290473938
Batch：245 | Loss: 0.5880527496337891
Batch：246 | Loss: 0.620638906955719
Batch：247 | Loss: 0.6035984754562378
Batch：248 | Loss: 0.5952376127243042
Batch：249 | Loss: 0.6073822975158691
Batch：250 | Loss: 0.6155564785003662
Batch：251 | Loss: 0.5900826454162598
Batch：252 | Loss: 0.6012860536575317
Batch：253 | Loss: 0.586810827255249
Batch：254 | Loss: 0.5885868668556213
Batch：255 | Loss: 0.5845849514007568
Batc

Batch：457 | Loss: 0.5191094279289246
Batch：458 | Loss: 0.5086175203323364
Batch：459 | Loss: 0.49493408203125
Batch：460 | Loss: 0.5203951597213745
Batch：461 | Loss: 0.5394601225852966
Batch：462 | Loss: 0.5152324438095093
Batch：463 | Loss: 0.5145617127418518
Batch：464 | Loss: 0.5167538523674011
Batch：465 | Loss: 0.4810032844543457
Batch：466 | Loss: 0.5065934658050537
Batch：467 | Loss: 0.5212317109107971
Batch：468 | Loss: 0.5207380056381226
Batch：469 | Loss: 0.519683837890625
Batch：470 | Loss: 0.5297590494155884
Batch：471 | Loss: 0.5312735438346863
Batch：472 | Loss: 0.5055584907531738
Batch：473 | Loss: 0.5427882671356201
Batch：474 | Loss: 0.5000283122062683
Batch：475 | Loss: 0.5273486971855164
Batch：476 | Loss: 0.5110967755317688
Batch：477 | Loss: 0.5111074447631836
Batch：478 | Loss: 0.5242206454277039
Batch：479 | Loss: 0.5102086663246155
Batch：480 | Loss: 0.5098307728767395
Batch：481 | Loss: 0.5120172500610352
Batch：482 | Loss: 0.5092320442199707
Batch：483 | Loss: 0.5319249629974365
Batc

Batch：679 | Loss: 0.4898419678211212
Batch：680 | Loss: 0.4430593252182007
Batch：681 | Loss: 0.42987191677093506
Batch：682 | Loss: 0.46600544452667236
Batch：683 | Loss: 0.4777469038963318
Batch：684 | Loss: 0.4810750186443329
Batch：685 | Loss: 0.45145025849342346
Batch：686 | Loss: 0.4679495096206665
Batch：687 | Loss: 0.46108654141426086
Batch：688 | Loss: 0.4676845669746399
Batch：689 | Loss: 0.46208614110946655
Batch：690 | Loss: 0.4525373578071594
Batch：691 | Loss: 0.4554130733013153
Batch：692 | Loss: 0.47062498331069946
Batch：693 | Loss: 0.4698718190193176
Batch：694 | Loss: 0.4503346383571625
Batch：695 | Loss: 0.4542345404624939
Batch：696 | Loss: 0.4530605375766754
Batch：697 | Loss: 0.4848295748233795
Batch：698 | Loss: 0.4368501305580139
Batch：699 | Loss: 0.4644647538661957
Batch：700 | Loss: 0.4477221965789795
Batch：701 | Loss: 0.46524015069007874
Batch：702 | Loss: 0.4589376449584961
Batch：703 | Loss: 0.4716220200061798
Batch：704 | Loss: 0.45000454783439636
Batch：705 | Loss: 0.4567628204

Batch：901 | Loss: 0.42370492219924927
Batch：902 | Loss: 0.3964269459247589
Batch：903 | Loss: 0.43700385093688965
Batch：904 | Loss: 0.40931737422943115
Batch：905 | Loss: 0.4212374985218048
Batch：906 | Loss: 0.4109145402908325
Batch：907 | Loss: 0.40279126167297363
Batch：908 | Loss: 0.43291234970092773
Batch：909 | Loss: 0.41975006461143494
Batch：910 | Loss: 0.41169074177742004
Batch：911 | Loss: 0.42123839259147644
Batch：912 | Loss: 0.41586023569107056
Batch：913 | Loss: 0.38403743505477905
Batch：914 | Loss: 0.4149063229560852
Batch：915 | Loss: 0.4227246940135956
Batch：916 | Loss: 0.413831502199173
Batch：917 | Loss: 0.4056617021560669
Batch：918 | Loss: 0.4051320254802704
Batch：919 | Loss: 0.4147071838378906
Batch：920 | Loss: 0.44267821311950684
Batch：921 | Loss: 0.40124088525772095
Batch：922 | Loss: 0.4161314368247986
Batch：923 | Loss: 0.41838133335113525
Batch：924 | Loss: 0.4182308614253998
Batch：925 | Loss: 0.4141588509082794
Batch：926 | Loss: 0.4290342926979065
Batch：927 | Loss: 0.409096

...

Batch：16811 | Loss: 0.04799111559987068
Batch：16812 | Loss: 0.03975357115268707
Batch：16813 | Loss: 0.04739416390657425
Batch：16814 | Loss: 0.049941159784793854
Batch：16815 | Loss: 0.048960160464048386
Batch：16816 | Loss: 0.0461307018995285
Batch：16817 | Loss: 0.052842069417238235
Batch：16818 | Loss: 0.05685187876224518
Batch：16819 | Loss: 0.04896882176399231
Batch：16820 | Loss: 0.04600830376148224
Batch：16821 | Loss: 0.04715147241950035
Batch：16822 | Loss: 0.05448330566287041
Batch：16823 | Loss: 0.04866552725434303
Batch：16824 | Loss: 0.04872046411037445
Batch：16825 | Loss: 0.0539713129401207
Batch：16826 | Loss: 0.058526407927274704
Batch：16827 | Loss: 0.05688212439417839
Batch：16828 | Loss: 0.04803144931793213
Batch：16829 | Loss: 0.05186181142926216
Batch：16830 | Loss: 0.05286567658185959
Batch：16831 | Loss: 0.04842371493577957
Batch：16832 | Loss: 0.05782194435596466
Batch：16833 | Loss: 0.05696165934205055
Batch：16834 | Loss: 0.04617103189229965
Batch：16835 | Loss: 0.0546432808041572

Batch：17018 | Loss: 0.04935111850500107
Batch：17019 | Loss: 0.05502120032906532
Batch：17020 | Loss: 0.04402296990156174
Batch：17021 | Loss: 0.047852952033281326
Batch：17022 | Loss: 0.05559432879090309
Batch：17023 | Loss: 0.045100849121809006
Batch：17024 | Loss: 0.05053401738405228
Batch：17025 | Loss: 0.04711877927184105
Batch：17026 | Loss: 0.05336228385567665
Batch：17027 | Loss: 0.03565547242760658
Batch：17028 | Loss: 0.052646931260824203
Batch：17029 | Loss: 0.05632340908050537
Batch：17030 | Loss: 0.048280566930770874
Batch：17031 | Loss: 0.06245812401175499
Batch：17032 | Loss: 0.04446648433804512
Batch：17033 | Loss: 0.04829736799001694
Batch：17034 | Loss: 0.045157358050346375
Batch：17035 | Loss: 0.060707151889801025
Batch：17036 | Loss: 0.0498201884329319
Batch：17037 | Loss: 0.048153653740882874
Batch：17038 | Loss: 0.05339732766151428
Batch：17039 | Loss: 0.03943457826972008
Batch：17040 | Loss: 0.04654502868652344
Batch：17041 | Loss: 0.04570968076586723
Batch：17042 | Loss: 0.049500741064

Batch：17226 | Loss: 0.046100739389657974
Batch：17227 | Loss: 0.03857358917593956
Batch：17228 | Loss: 0.04829000309109688
Batch：17229 | Loss: 0.045424703508615494
Batch：17230 | Loss: 0.060945842415094376
Batch：17231 | Loss: 0.049002114683389664
Batch：17232 | Loss: 0.04734312370419502
Batch：17233 | Loss: 0.04588593542575836
Batch：17234 | Loss: 0.04614534601569176
Batch：17235 | Loss: 0.05417880043387413
Batch：17236 | Loss: 0.04755304381251335
Batch：17237 | Loss: 0.04649451747536659
Batch：17238 | Loss: 0.0469561442732811
Batch：17239 | Loss: 0.049427714198827744
Batch：17240 | Loss: 0.03828795999288559
Batch：17241 | Loss: 0.05860419198870659
Batch：17242 | Loss: 0.04676011949777603
Batch：17243 | Loss: 0.05476173013448715
Batch：17244 | Loss: 0.045798856765031815
Batch：17245 | Loss: 0.056345485150814056
Batch：17246 | Loss: 0.042248040437698364
Batch：17247 | Loss: 0.04959169402718544
Batch：17248 | Loss: 0.049163736402988434
Batch：17249 | Loss: 0.04726876690983772
Batch：17250 | Loss: 0.0484369397

Batch：17434 | Loss: 0.04447963461279869
Batch：17435 | Loss: 0.049389082938432693
Batch：17436 | Loss: 0.04811195284128189
Batch：17437 | Loss: 0.04072337597608566
Batch：17438 | Loss: 0.04433668032288551
Batch：17439 | Loss: 0.049897387623786926
Batch：17440 | Loss: 0.0408644825220108
Batch：17441 | Loss: 0.05432410538196564
Batch：17442 | Loss: 0.043965380638837814
Batch：17443 | Loss: 0.05408589169383049
Batch：17444 | Loss: 0.04236462339758873
Batch：17445 | Loss: 0.03792206943035126
Batch：17446 | Loss: 0.04987972974777222
Batch：17447 | Loss: 0.047718048095703125
Batch：17448 | Loss: 0.056390970945358276
Batch：17449 | Loss: 0.050664469599723816
Batch：17450 | Loss: 0.0506933368742466
Batch：17451 | Loss: 0.04810728877782822
Batch：17452 | Loss: 0.0521053671836853
Batch：17453 | Loss: 0.04317699000239372
Batch：17454 | Loss: 0.050702158361673355
Batch：17455 | Loss: 0.04568443074822426
Batch：17456 | Loss: 0.043824829161167145
Batch：17457 | Loss: 0.0475710891187191
Batch：17458 | Loss: 0.05084637552499

0.13338477108401223

In [17]:
torch.save(model0.state_dict(), 'ae01.pt')

In [18]:
model0.load_state_dict(torch.load('ae01.pt'))

<All keys matched successfully>

In [18]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [89]:
N_EPOCHS = 3
CLIP = 1

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model0, trainiter, optimizer, criterion, CLIP,  device, maxlen)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
 
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')

Epoch: 01 | Time: 5m 49s
	Train Loss: 0.038 | Train PPL:   1.039
Epoch: 02 | Time: 5m 52s
	Train Loss: 0.028 | Train PPL:   1.028
Epoch: 03 | Time: 5m 43s
	Train Loss: 0.022 | Train PPL:   1.023


In [91]:
torch.save(model0.state_dict(), 'aep4.pt')

In [ ]:
torch.save(model10.state_dict(), 'ae01.pt')

In [19]:
model0.load_state_dict(torch.load('aep4.pt'))

<All keys matched successfully>

In [20]:
# Start inferencing
# data cannot be shuffled, build new loader
evl = {'batch_size': 256,
          'shuffle': False}

filteriter = DataLoader(train_X, **evl)

In [21]:
criterionf  = nn.MSELoss(reduction='none')

In [22]:
def flt(model, iterator, criterion, device, maxlen):
    
    model.eval()
    
    losst = np.zeros(1)
           
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            tin = batch.to(device)

            target, recons = model(tin, maxlen)
            
            loss = torch.sum(criterionf(target,recons),[1,2])
            
            losst = np.append(losst,loss.cpu().numpy())
        
    return losst

In [40]:
losst = flt(model0, filteriter, criterionf, device, maxlen)

In [41]:
error = losst[1:]

In [42]:
error = pd.DataFrame(error, index=traintitle.index.values)

In [43]:
error

,0
0,292.566956
2,0.060529
4,0.044393
5,244.851639
6,217.494537
...,...
2344431,214.971603
2344432,0.419960
2344433,423.000916
2344434,169.982880


In [53]:
toshow = error.sort_values(by=[0],ascending = False)[0:50].index.values

In [47]:
toshow

array([1756484, 2334546, 1694752,  677822,  318932, 1465770,  232808,
        285304,  331794,  318186])

In [54]:
traintitle.loc[toshow]

1756484    brainwashing begin brainwashing begin real tal...
2334546    sfesgrtugiyhi rer5t6y7hu8hjy7e##rt6y##ujhy7gt#...
1694752    kirtasi̇ye kopyalama baski kopyalama sektörün ...
677822     promo code free io app download download tell ...
318932     akcesoria elektryknet poszukujesz porządnie wy...
1465770    dbhfyjnn dhdrhjf ddhddwe sdd sddss shfgd hsbee...
232808     nakit ogledlce nakit online ogledalce nudi vel...
285304     oryginalna srebrna nasze produkty autorskie po...
331794     http katalog budowlany budowlane przetargi bud...
318186     puertas abatibles puertas abatibles fabricacio...
319994     monday development stolica wielkopolski szybko...
1105807    decryption challenge dwi##tytyfn8yqv+3mo2i+1we...
1419643    unawezaje kumridhisha mpenzi nimekuwa kwenye m...
1463853    meetup use massively long identifier email ver...
298472     ray dolap mobilya tasarımında firmalar arasınd...
167662     halı altı camii altı karbon film sektöründe hi...
111711     demonte prefa